In [ ]:
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
import plotly.io as pio
from sqlalchemy import create_engine
import logging
import os

db_url = os.getenv("DATABASE_URL")
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
pd.set_option("display.float_format", "{:.2f}".format)

In [ ]:
# Connect to your Postgres database
engine = create_engine(db_url)

# Load the table into a DataFrame
df = pd.read_sql("SELECT * FROM houston_311", engine)

pio.renderers.default = "notebook_connected"
df.head()

/var/folders/_v/f9jlhvnd2xg9kybwfc9yh6nw0000gn/T/ipykernel_36887/2006162025.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


,CASE NUMBER,NEIGHBORHOOD,DEPARTMENT,DIVISION,CASE TYPE,CREATED DATE,CLOSED DATE,LATITUDE,LONGITUDE,CATEGORY
0,12091834-101002444724,EAST LITTLE YORK / HOMESTEAD,PWE Public Works Engineering,PU Public Utilities,Fire Hydrant,2017-01-01 00:01:48,2017-01-01 11:20:02,29.856743,-95.296945,Water & Sewer
1,12091835-101002444725,NORTHSIDE/NORTHLINE,PWE Public Works Engineering,PU Public Utilities,Fire Hydrant,2017-01-01 00:07:29,2017-01-01 04:50:02,29.830449,-95.367947,Water & Sewer
2,101002444726,MID WEST,PWE Public Works Engineering,Traffic Operations,Traffic Signal Maintenance,2017-01-01 00:14:04,2017-01-01 00:23:57,29.722099,-95.539078,Traffic & Roads
3,101002444727,WASHINGTON AVENUE COALITION / MEMORIAL P,PWE Public Works Engineering,PDS Planning Development Services,MultiFamily Habitability Violation,2017-01-01 00:17:23,2017-01-04 14:01:37,29.773859,-95.402572,Other
4,12091836-101002444730,GREATER UPTOWN,PWE Public Works Engineering,PU Public Utilities,Water Leak,2017-01-01 00:30:48,2017-02-02 13:20:06,29.733522,-95.466104,Water & Sewer


In [3]:
#converting dates to datetime format
date_cols = ["CREATED DATE", "CLOSED DATE"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")  # invalid : NaT

In [4]:
#checking for missing closed dates
missing_resolved = df[df['CLOSED DATE'].isna()]
print(len(missing_resolved))
missing_resolved.head()

212225


,CASE NUMBER,NEIGHBORHOOD,DEPARTMENT,DIVISION,CASE TYPE,CREATED DATE,CLOSED DATE,LATITUDE,LONGITUDE,CATEGORY
61837,101002517814,ACRES HOME,311 HelpLine,Liaison Assistance,Liaison Note,2017-03-01 15:11:29,NaT,29.844591,-95.440855,Mayor's Office
72338,101002530698,WASHINGTON AVENUE COALITION / MEMORIAL P,311 HelpLine,Liaison Assistance,Liaison Note,2017-03-13 14:38:21,NaT,29.772592,-95.371434,Mayor's Office
119776,101002587201,ELDRIDGE / WEST OAKS,311 HelpLine,Liaison Assistance,Liaison Note,2017-05-01 11:19:17,NaT,29.716226,-95.612038,Mayor's Office
145001,101002617459,AFTON OAKS / RIVER OAKS AREA,311 HelpLine,Liaison Assistance,Liaison Note,2017-05-30 07:49:49,NaT,29.746519,-95.433993,Mayor's Office
218430,101002704773,NORTHSIDE/NORTHLINE,311 HelpLine,Liaison Assistance,Identify Numbered Address,2017-08-12 15:09:51,NaT,29.845793,-95.366716,Health & Safety


In [5]:
valid_df = df.dropna(subset=['CLOSED DATE'])
valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days
avg_resolution = (
    valid_df.groupby('CATEGORY')['RESOLUTION TIME']
    .mean()
    .reset_index(name='Avg_Resolution_Days')
    .sort_values('Avg_Resolution_Days')
)

print(avg_resolution)

               CATEGORY  Avg_Resolution_Days
8       Traffic & Roads             8.670631
10        Water & Sewer             9.363514
9       Trash / Garbage            17.703599
3        Mayor's Office            26.512100
4                 Other            28.021725
0         Admin / Legal            28.800884
7    Storms / Disasters            29.439805
6             Recycling            33.960988
1       Health & Safety            50.933181
5         Parks & Trees            51.078343
2   Housing & Buildings            54.645367


/var/folders/_v/f9jlhvnd2xg9kybwfc9yh6nw0000gn/T/ipykernel_36887/1969648791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days


In [8]:
valid_df = df.dropna(subset=['CLOSED DATE'])
valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days
avg_resolution = (
    valid_df.groupby('NEIGHBORHOOD')['RESOLUTION TIME']
    .mean()
    .reset_index(name='Avg_Resolution_Days')
    .sort_values('Avg_Resolution_Days')
)

print(avg_resolution)

                  NEIGHBORHOOD  Avg_Resolution_Days
42      Greater Greenspoint MD             0.000000
25                   FB CAD #6             2.000000
7              BRAESWOOD PLACE             8.912621
10            BRIARFOREST AREA            11.149425
58         MEDICAL CENTER AREA            12.606714
..                         ...                  ...
76                   SETTEGAST            27.238565
87   TRINITY / HOUSTON GARDENS            27.765826
15  CLINTON PARK TRI-COMMUNITY            29.313805
31          GREATER FIFTH WARD            29.494319
65              NEAR NORTHSIDE            30.839311

[97 rows x 2 columns]


/var/folders/_v/f9jlhvnd2xg9kybwfc9yh6nw0000gn/T/ipykernel_36887/139839970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days


In [7]:
valid_df = df.dropna(subset=['CLOSED DATE'])
valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days
avg_resolution = (
    valid_df.groupby('DIVISION')['RESOLUTION TIME']
    .mean()
    .reset_index(name='Avg_Resolution_Days')
    .sort_values('Avg_Resolution_Days')
)

print(avg_resolution)

                              DIVISION  Avg_Resolution_Days
13                Emergency Evacuation             1.515625
12         EC Engineering Construction             1.842188
37   PDS Planning Development Services             2.081728
23      Government Relationship Office             2.250000
34           Mayor's Assistance Office             2.776248
30                               Legal             2.800000
0                    311 Call Handling             3.041585
39                 PU Public Utilities             4.015910
2         Capitol Improvement Planning             5.530890
17       FM Facilities and Maintenance             5.898389
41           Parking Meter Maintenance             6.348945
36                           Occupancy             6.573904
52                  Traffic Operations             6.650648
51                 Street and Drainage             7.263996
40                 Parking Enforcement             8.949390
19                     Fire Prevention  

/var/folders/_v/f9jlhvnd2xg9kybwfc9yh6nw0000gn/T/ipykernel_36887/3320271243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days


valid_df = df.dropna(subset=['CLOSED DATE'])
valid_df['RESOLUTION TIME'] = (valid_df['CLOSED DATE'] - valid_df['CREATED DATE']).dt.days
avg_resolution = (
    valid_df.groupby('DEPARTMENT')['RESOLUTION TIME']
    .mean()
    .reset_index(name='Avg_Resolution_Days')
    .sort_values('Avg_Resolution_Days')
)

print(avg_resolution)